In [1]:
#95% confidence interval for the Bayesian population mean change on the first therapy
import numpy as np 
import pandas as pd 
from scipy.stats import t
Anor = pd.read_csv("Anorexia.dat", sep = '\s+') 
diff = Anor['after'] - Anor['before']
Anor['diff'] = diff
first = Anor.loc[Anor['therapy'] == 'cb']['diff']
m1 = first.mean(); n1 = len(first); s1 = first.std()
first_posterior = t(loc = m1, scale = s1/np.sqrt(n1), df = n1-1) 
first_posterior.interval(0.95)

(0.22689015835877635, 5.7869029450894995)

In [2]:
#99% confidence interval for the Bayesian population mean change on the first therapy
diff = Anor['after'] - Anor['before']
Anor['diff'] = diff
third = Anor.loc[Anor['therapy'] == 'c']['diff']
m3 = third.mean(); n3 = len(third); s3 = third.std() 
third_posterior = t(loc = m3, scale = s3/np.sqrt(n3), df = n3-1) 
third_posterior.interval(0.99)

(-4.817114312983953, 3.9171143129839545)

In [2]:
#The 2018 General Social Survey (GSS) in the U.S. asked “Do you believe in hell?”
#Of the 1142 respondents, 814 said yes and 328 said no.
#The ML estimate of π is πˆ = 814/1142 = 0.713,
#and you can check that the 95% score confidence interval (4.7) is (0.686, 0.738). 
#How does this compare to a Bayesian estimate and posterior interval?
from statsmodels.stats.proportion import proportion_confint
proportion_confint(814, 1142, method='jeffreys')

import pymc3
from scipy.stats import beta
beta_dist = beta.rvs(size = 5000000, a = 814.5, b = 328.5)
print(pymc3.stats.hdi(beta_dist, alpha=0.05))

import numpy as np
print('[',np.quantile(beta_dist, 0.025),',', np.quantile(beta_dist, 0.975),']')



[0.68735461 0.73767268]
[ 0.6860382553455946 , 0.7384796012256739 ]


In [5]:
from pymc3 import  *
import numpy as np
import pandas as pd

patient = pd.read_csv('Anorexia.dat', sep='\s+')
diff = patient['after'] - patient['before']
patient['diff'] = diff
print(patient.loc[patient['therapy'] == 'cb']['diff'].describe())
diffCB = patient.loc[patient['therapy'] == 'cb']['diff']

data = dict(y = diffCB)
B0=10**(-7)
with Model() as model:
# define highly disperse priors for variance and mean
    sigma = InverseGamma('sigma', B0, B0, testval=1.)
    intercept = Normal('Intercept', 0, sigma=1/B0)
# define likelihood function for normal responses
    likelihood = Normal('y',mu=intercept,sigma=sigma,observed=diffCB)
    trace = sample(50000, cores=2)  # 100000 posterior samples
np.mean(trace['Intercept'])

np.std(trace['Intercept'])
pymc3.stats.hdi(trace['Intercept'], alpha=0.05)



count    29.000000
mean      3.006897
std       7.308504
min      -9.100000
25%      -0.700000
50%       1.400000
75%       3.900000
max      20.900000
Name: diff, dtype: float64


/Users/jessicawin/opt/anaconda3/lib/python3.9/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [Intercept, sigma]


/Users/jessicawin/opt/anaconda3/lib/python3.9/site-packages/scipy/stats/_continuous_distns.py:624: RuntimeWarning: overflow encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
Sampling 2 chains for 1_000 tune and 50_000 draw iterations (2_000 + 100_000 draws total) took 25 seconds.


array([0.27422238, 5.6045742 ])